<a href="https://colab.research.google.com/github/maxlc123/iphs_290_cultural_analytics_lda_topic_modeling/blob/main/Copy_of_nlp_lda_topic_modeling_gensim_20221204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [ ]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 3.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=9e0cc711c04f1d2d3eaaacbcad2eef99bd774f7038f74fb24cc742c2b7313c82
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=e85a8ae641e97434dbd6f9d8928648eb7756b656195bc864c541ee27789994ab
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 4.2 MB/s 
     |████████████████████████████████| 287 kB 31.1 MB/s 


## Configure Jupyter Notebook

In [ ]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from slugify import slugify

# Get Text Data

You have (2) ways to get data in this tutorial, but if you're following
this tutorial just to learn about LDA I encourage you to consider picking a
corpus on a subject that you are familiar with. Qualitatively evaluating the
output of an LDA model is challenging and can require you to understand the
subject matter of your corpus (depending on your goal with the model).

Reference to Compare::

    The NeurIPS corpus contains 1740 documents, and not particularly long ones.

    `website <http://www.cs.nyu.edu/~roweis/data.html>`
    
    So keep in mind that this tutorial is not geared towards efficiency, and be
    careful before applying the code to a large dataset.


## Upload Raw Textfile

In [ ]:
from google.colab import files

TEXT_ENCODING = 'utf-8'

### OPTION (i): Novels

In [ ]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

Saving TheGreatGatsby_FScottFitzgerald.txt to TheGreatGatsby_FScottFitzgerald.txt
User uploaded file "TheGreatGatsby_FScottFitzgerald.txt" with length 274704 bytes
CPU times: user 745 ms, sys: 77.5 ms, total: 822 ms
Wall time: 1min 5s


In [ ]:
text_raw_str[:500]

'Chapter 1\r\n\r\n\r\n\r\nIn my younger and more vulnerable years my father gave me some advice\r\nthat I\'ve been turning over in my mind ever since.\r\n\r\n"Whenever you feel like criticizing any one," he told me, "just\r\nremember that all the people in this world haven\'t had the advantages\r\nthat you\'ve had."\r\n\r\nHe didn\'t say any more but we\'ve always been unusually communicative\r\nin a reserved way, and I understood that he meant a great deal more\r\nthan that. In consequence I\'m inclined to reserve all judgment'

### OPTION (ii): Tweets

In [ ]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

KeyError: ignored

In [ ]:
# Recover parameters of original search from filename components
word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

print(f'word1: {word1}')
print(f'word2: {word2}')
print(f'search_str: {search_str}')
print(f'start_date: {start_date}')
print(f'end_date: {end_date}')


word1: tweets
word2: raw
search_str: twitter-acquisition
start_date: 2022-11-07
end_date: 2022-12-06


In [ ]:
# TWITTER

# Read raw tweet datafile into DataFrame 'data'

try:
  headers = ['datetime','id','tweet','username','reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id',
             'text_raw','text_noemoji','text_clean','vader','textblob','distilbert'] # ,'roberta']
  dtypes = {'datetime':'str',
            'id':'int',
            'tweet':'string',
            'username':'string',
            'reply_ct':'int',
            'retweet_ct':'int',
            'like_ct':'int',
            'quote_ct':'int',
            'retweeted_id':'string',
            'quoted_id':'string',
            'text_raw':'string',
            'text_noemoji':'string',
            'text_clean':'string',
            'vader':'float',
            'textblob':'float',
            'distilbert':'float'}
            # 'roberta':'float'}
  parse_dates = ['datetime']
  text_df = pd.read_csv(text_filename_str, skiprows=[0], names=headers, dtype=dtypes, parse_dates=parse_dates, index_col=None)
  text_df.head()
  text_df.info()
except:
  print(f'ERROR: Tweets datafile cannot be read by pd.read_csv')

,datetime,id,tweet,username,reply_ct,retweet_ct,like_ct,quote_ct,retweeted_id,quoted_id,text_raw,text_noemoji,text_clean,vader,textblob,distilbert
0,2022-11-07 12:52:15+00:00,1589601624531423233,"I love how we add ""gate"" to every scandal out ...",DirtyDryIce,0,0,0,0,<NA>,<NA>,"I love how we add ""gate"" to every scandal out ...","I love how we add "" gate "" to every scandal ou...",love add gate scandal matter insignificant wan...,NaN,NaN,NaN
1,2022-11-07 12:53:27+00:00,1589601928870514690,the thing I find most hilarious about a migrat...,elfexgirlfriend,1,0,4,0,<NA>,<NA>,the thing I find most hilarious about a migrat...,the thing I find most hilarious about a migrat...,thing find hilarious migration tumblr half fri...,NaN,NaN,NaN
2,2022-11-07 12:53:28+00:00,1589601932364361728,While elon musk said 'comedy is now legal' fol...,psalm7115,0,0,1,0,<NA>,<NA>,While elon musk said 'comedy is now legal' fol...,While elon musk said ' comedy is now legal ' f...,elon musk say comedy legal follow acquisition ...,NaN,NaN,NaN
3,2022-11-07 12:54:35+00:00,1589602211533066246,Musk’s $44 billion acquisition deal piled Twit...,AnneHandler,0,0,0,0,<NA>,<NA>,Musk’s $44 billion acquisition deal piled Twit...,Musk ' s $ 44 billion acquisition deal piled T...,musk s billion acquisition deal pile twitter b...,NaN,NaN,NaN
4,2022-11-07 12:56:12+00:00,1589602618015449090,EsensConsulting: #Decentralized : #Mastodon re...,Guillaume_GCH,0,0,0,1,<NA>,<NA>,EsensConsulting: #Decentralized : #Mastodon re...,EsensConsulting #Decentralized #Mastodon recor...,esensconsulting decentralize mastodon record k...,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   datetime      10000 non-null  datetime64[ns, UTC]
 1   id            10000 non-null  int64              
 2   tweet         10000 non-null  string             
 3   username      10000 non-null  string             
 4   reply_ct      10000 non-null  int64              
 5   retweet_ct    10000 non-null  int64              
 6   like_ct       10000 non-null  int64              
 7   quote_ct      10000 non-null  int64              
 8   retweeted_id  0 non-null      string             
 9   quoted_id     701 non-null    string             
 10  text_raw      10000 non-null  string             
 11  text_noemoji  10000 non-null  string             
 12  text_clean    10000 non-null  string             
 13  vader         0 non-null      float64            
 14  textblo

In [ ]:
# Reform Tweets to assume the form of a Novel

line_ls = []

for i, aline in enumerate(text_df.text_clean.to_list()):
  if (i % 100) == 0:
    print(f'Line #{i}: {aline}')
  line_ls.append(aline)

print('\n\n')

text_raw_str = "\n\n".join(line_ls)
text_raw_str[:500]


Line #0: love add gate scandal matter insignificant want twitter acquisition elongate come dangerously close give compliment
Line #100: musk twitter acquisition mean social medium crypto adoption url finance fintech cryptocurrencie
Line #200: don t disclose actual acquisition
Line #300: support russian invasion way japan timeline calm acquisition japanese talk change issue war originally twitter japanese
Line #400: face tear joy face tear joy face tear joy face tear joy go track twitter acquisition arrogance overconfidence fine line
Line #500: real agenda musk s acquisition twitter shill trump
Line #600: elon musk news acquisition twitter likely save democracy damage war elite win t reply text message url
Line #700: elon musk s twitter acquisition affect midterm url
Line #800: joke elon musk acquisition twitter fresh wind sail dogecoin meme cryptocurrency catapult fame url
Line #900: elon musk twitter acquisition affect midterm url url
Line #1000: want delete tweet remove bunch tweet p

'love add gate scandal matter insignificant want twitter acquisition elongate come dangerously close give compliment\n\nthing find hilarious migration tumblr half friend leave tumblr twitter unfortunate acquisition subsequent purging nsfw content appealing advertiser\n\nelon musk say comedy legal follow acquisition twitter joke new owner criticism takeover get user block suspend url\n\nmusk s billion acquisition deal pile twitter billion debt result company have pay billion year interest expense despit'

# Enter Title

In [ ]:
# If this is a Twitter textfile, the search terms are recovered from the filename

# Recover parameters of original search from filename components
try:
  word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

  print(f'word1: {word1}')
  print(f'word2: {word2}')
  print(f'search_str: {search_str}')
  print(f'start_date: {start_date}')
  print(f'end_date: {end_date}')
  print('\n')
  
  text_title_str = f'[{search_str}] from [{start_date}] to [{end_date}]'
  print(f'\nTwitter Search String:\n\n  {text_title_str}\n')

except:
  print('This is not a Twitter datafile named in standard format\n\nEnter the Novel information below.')

This is not a Twitter datafile named in standard format

Enter the Novel information below.


In [ ]:
#@title Enter Title "[Title] by [Author]""

Text_Title = "The Great Gatsby by F. Scott Fitzgerald" #@param {type:"string"}

# Remove illegal filename punctuation
Text_Title = Text_Title.replace("'","")
Text_Title = Text_Title.replace('"','')
Text_Title = Text_Title.replace('.',' ')

text_title_str = ' '.join([x.capitalize() for x in slugify(Text_Title).replace('-',' ').split()])
text_title_str

'The Great Gatsby By F Scott Fitzgerald'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Segment Text

In [ ]:
text_raw_str[:500]

'Chapter 1\r\n\r\n\r\n\r\nIn my younger and more vulnerable years my father gave me some advice\r\nthat I\'ve been turning over in my mind ever since.\r\n\r\n"Whenever you feel like criticizing any one," he told me, "just\r\nremember that all the people in this world haven\'t had the advantages\r\nthat you\'ve had."\r\n\r\nHe didn\'t say any more but we\'ve always been unusually communicative\r\nin a reserved way, and I understood that he meant a great deal more\r\nthan that. In consequence I\'m inclined to reserve all judgment'

In [ ]:
# Split into paragraphs
text_parags_ls = text_raw_str.split('\n')
print(f'There are {len(text_parags_ls)} raw paragraphs\n')

# Delete empty paragraphs
text_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]
print(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')

# Replace stray/embedded /n with a space
text_parags_clean_ls = []
for aparag in text_parags_ls:
  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())

text_parags_ls = text_parags_clean_ls
text_parags_ls[:5]

There are 6319 raw paragraphs

There are 4628 non-empty paragraphs




['Chapter 1',
 'In my younger and more vulnerable years my father gave me some advice',
 "that I've been turning over in my mind ever since.",
 '"Whenever you feel like criticizing any one," he told me, "just',
 "remember that all the people in this world haven't had the advantages"]

In [ ]:
# Verify first paragraph (after title and chapter headings)

text_parags_ls[3]
print('\n')
# Paragraph char count
len(text_parags_ls[3])

'"Whenever you feel like criticizing any one," he told me, "just'

63

In [ ]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 1000

# Delete any paragraphs shorter than MIN_LEN_PARAG
text_parags_ls = [x for x in text_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
text_parags_ls = [' '.join(x.split()) for x in text_parags_ls]

len(text_parags_ls)

4603

In [ ]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars

parag_ct = len(text_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = text_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')

There are now 253 Documents of 1000 chars or more


In [ ]:
# View the first 5 docs

docs[:5]

['Chapter 1In my younger and more vulnerable years my father gave me some advicethat I\'ve been turning over in my mind ever since."Whenever you feel like criticizing any one," he told me, "justremember that all the people in this world haven\'t had the advantagesthat you\'ve had."He didn\'t say any more but we\'ve always been unusually communicativein a reserved way, and I understood that he meant a great deal morethan that. In consequence I\'m inclined to reserve all judgments,a habit that has opened up many curious natures to me and alsomade me the victim of not a few veteran bores. The abnormal mindis quick to detect and attach itself to this quality when itappears in a normal person, and so it came about that in college Iwas unjustly accused of being a politician, because I was privy to thesecret griefs of wild, unknown men. Most of the confidences wereunsought--frequently I have feigned sleep, preoccupation, or a hostilelevity when I realized by some unmistakable sign that an int

In [ ]:
# View the first 500 chars in the 50th doc

docs[49][:500]

'even told me about it, and the man came after it one day when he was out.She looked around to see who was listening: " \'Oh, is that your suit?\' I\'This is the first I ever heard about it.\' But I gave it to him and then Ilay downand cried to beat the band all afternoon.""She really ought to get away from him," resumed Catherine to me."They\'ve been living over that garage for eleven years. And Tom\'s thefirst sweetie she ever had."The bottle of whiskey--a second one--was now in constant demand by al'

In [ ]:
# View the last 100 chars in the last doc

docs[-1][-100:]

'nd one finemorning----So we beat on, boats against the current, borne back ceaselessly intothe past.'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Steps

#### Expand Contractions

In [ ]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
contractions.fix("yall're happy now it isn't a don't tell problem", slang=False) # default: true

"yall're happy now it is not a do not tell problem"

In [ ]:
# Expand all Contractions paragraph by paragraph
docs_clean_ls = []
for adoc in docs:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]

['Chapter 1In my younger and more vulnerable years my father gave me some advicethat I have been turning over in my mind ever since."Whenever you feel like criticizing any one," he told me, "justremember that all the people in this world have not had the advantagesthat you have had."He did not say any more but we have always been unusually communicativein a reserved way, and I understood that he meant a great deal morethan that. In consequence I am inclined to reserve all judgments,a habit that has opened up many curious natures to me and alsomade me the victim of not a few veteran bores. The abnormal mindis quick to detect and attach itself to this quality when itappears in a normal person, and so it came about that in college Iwas unjustly accused of being a politician, because I was privy to thesecret griefs of wild, unknown men. Most of the confidences wereunsought--frequently I have feigned sleep, preoccupation, or a hostilelevity when I realized by some unmistakable sign that an 

#### Tokenize Text

In [ ]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [ ]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

['chapter',
 '1in',
 'my',
 'younger',
 'and',
 'more',
 'vulnerable',
 'years',
 'my',
 'father',
 'gave',
 'me',
 'some',
 'advicethat',
 'have',
 'been',
 'turning',
 'over',
 'in',
 'my',
 'mind',
 'ever',
 'since',
 'whenever',
 'you',
 'feel',
 'like',
 'criticizing',
 'any',
 'one',
 'he',
 'told',
 'me',
 'justremember',
 'that',
 'all',
 'the',
 'people',
 'in',
 'this',
 'world',
 'have',
 'not',
 'had',
 'the',
 'advantagesthat',
 'you',
 'have',
 'had',
 'he',
 'did',
 'not',
 'say',
 'any',
 'more',
 'but',
 'we',
 'have',
 'always',
 'been',
 'unusually',
 'communicativein',
 'reserved',
 'way',
 'and',
 'understood',
 'that',
 'he',
 'meant',
 'great',
 'deal',
 'morethan',
 'that',
 'in',
 'consequence',
 'am',
 'inclined',
 'to',
 'reserve',
 'all',
 'judgments',
 'habit',
 'that',
 'has',
 'opened',
 'up',
 'many',
 'curious',
 'natures',
 'to',
 'me',
 'and',
 'alsomade',
 'me',
 'the',
 'victim',
 'of',
 'not',
 'few',
 'veteran',
 'bores',
 'the',
 'abnormal',
 'mi



There are 172 tokens in the first document


#### Customize Stopwords

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [ ]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopwords_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')

['herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 



There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [ ]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


#### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 1.63 s, sys: 71.8 ms, total: 1.7 s
Wall time: 1.7 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [ ]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 5 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.49 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [ ]:
# docs_ls_ls = [astring.split() for astring in docs]
# docs = docs_ls_ls

In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [ ]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

253

In [ ]:
type(corpus[0])

list

In [ ]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 1),
 (9, 1)]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 964
Number of documents: 253


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [ ]:
#@markdown How many Topics do you want to find?

No_of_Topics = 10 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [ ]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 3.39 s, sys: 46.2 ms, total: 3.44 s
Wall time: 3.46 s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)


# from pprint import pprint
# pprint(top_topics)

# print('\n\n')
# for i,atopic in enumerate(top_topics):
#   atopic_coherence_str = f'Topic #{i}: coherence = {atopic[1]}'
#   print(atopic_coherence_str)
#   report_line_ls.append(atopic_coherence_str)


topic_word_ls = []

topic_word_ls.append(text_title_str)

# print(text_title_str)

for i, atopic in enumerate(top_topics):
  topic_str = f'\n\nTopic #{i}: {atopic[1]} (coherence)'
  # print(topic_str)
  topic_word_ls.append(topic_str)
  for j, aword in enumerate(atopic[0]):
    word_str = f'  Word #{j}: {aword}'
    # print(word_str)
    topic_word_ls.append(word_str)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
avg_topic_coherence_str = f'Average topic coherence: {avg_topic_coherence:.4f}'
# print(avg_topic_coherence_str)
topic_word_ls.append(f'\n\n{avg_topic_coherence_str}')

topic_word_str = "\n".join(topic_word_ls)
print(topic_word_str)

# Save to file
filename_topic_word_str = f'lda_topic_word_report.txt'
with open(filename_topic_word_str, 'w') as fp:
  fp.write(topic_word_str)

files.download(filename_topic_word_str)

The Great Gatsby By F Scott Fitzgerald


Topic #0: -0.9222677762928143 (coherence)
  Word #0: (0.011974162, 'tom')
  Word #1: (0.01158095, 'do')
  Word #2: (0.010578074, 'we')
  Word #3: (0.010524663, 'what')
  Word #4: (0.0094726, 'did')
  Word #5: (0.009149128, 'have')
  Word #6: (0.008833581, 'if')
  Word #7: (0.00844679, 'daisy')
  Word #8: (0.0082881, 'this')
  Word #9: (0.007726801, 'would')
  Word #10: (0.0077033187, 'an')
  Word #11: (0.007571487, 'one')
  Word #12: (0.007053175, 'so')
  Word #13: (0.006837032, 'are')
  Word #14: (0.0067437934, 'know')
  Word #15: (0.006615188, 'about')
  Word #16: (0.0064966637, 'like')
  Word #17: (0.006307618, 'were')
  Word #18: (0.0060513145, 'man')
  Word #19: (0.006042431, 'then')


Topic #1: -0.972677527839028 (coherence)
  Word #0: (0.011273947, 'daisy')
  Word #1: (0.011114904, 'tom')
  Word #2: (0.009186996, 'over')
  Word #3: (0.008894915, 'were')
  Word #4: (0.0085534705, 'an')
  Word #5: (0.008040821, 'we')
  Word #6: (0.008024557

7159

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [ ]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


